# Replacing the UK personal allowance with a universal basic income

## This budget-neutral proposal could decrease poverty by up to 29%.

Among several potential pay-fors in their recent [UBI discussion paper](https://d3n8a8pro7vhmx.cloudfront.net/libdems/pages/1811/attachments/original/1621669347/145_-_Universal_Basic_Income.docx_%281%29.pdf?1621669347), the Liberal Democrats considered lowering the personal allowance from its current value of £12,570 to £2,500. That is, the first £2,500 of an individual's annual earnings would be exempted from taxation, rather than the first £12,570. [As my colleague Nikhil Woodruff found](https://www.ubicenter.org/lib-dem-policy-paper), this would raise about £60 billion which, when funding a UBI, significantly lowers poverty and inequality.

The Liberal Democrats were not the first to connect the personal allowance and UBI. The Green Party has also proposed effectively exchanging the two policies[^1], and several think tanks have modeled UBIs involving reductions or repeals of the personal allowance. While the personal allowance is highly salient— a 2014 poll found that [85% of the public](https://www.ipsos.com/ipsos-mori/en-uk/personal-allowances-rise-most-popular-conference-season-tax-pledges) supported raising it—it's also a large source of potential revenue for the costly proposition of paying every member of society every month.

In this analysis, I extend the literature around the personal allowance and UBI by modeling a range of personal allowance values and outcomes. I find that repealing the personal allowance could fund a £29 weekly UBI to all members of society, including children. This reform would cut the overall poverty rate by 29%, deep poverty by 46%, and child poverty by 51%. I also find that personal allowance reductions matter on the margin: each £2,000 cut, when funding a UBI, yields around a 5% decrease in poverty. 

[^1]: The Green Party states that in their model, UBI will be taxable, but ["all income tax payers will have a tax-free allowance which is the equivalent to their Universal Basic Income amount"](https://www.greenparty.ie/wp-content/uploads/2018/07/Green-Party-Universal-Basic-Income-Policy.pdf); this means that, in practice, the UBI would not be taxed and the personal allowance would effectively be eliminated.  

### How the personal allowance works


The personal allowance was [introduced in 1979](http://taxhistory.co.uk/Income%20Tax%20Allowances.htm), and it has grown almost every year since. Liberal Democrats accelerated its growth by [calling](https://www.standard.co.uk/news/politics/libdems-to-let-1-3m-low-earners-avoid-paying-tax-8506838.html) for expanding it from £10,000 in 2015 to £12,500 in 2020; this succeeded, and it now sits at £12,570, where it is [expected to remain until April 2026](https://www.reuters.com/article/uk-health-coronavirus-britain-budget-inc-idUSKBN2AV1LP).

While many countries have a tax-free band like the personal allowance, the UK also makes the more uncommon decision to phase it out for higher earners. Individuals earning above £100,000 lose £1 of personal allowance for every £2 in earnings, phasing out fully for those earning £125,140 per year and creating 62% marginal tax rates for earners in this range.

These components combine to produce a policy that is neither clearly progressive nor clearly regressive. Because households in the bottom decile have little to no earned income, the personal allowance provides them only small benefits. And while those in the upper individual decile are mostly excluded from the personal allowance, those in the upper household decile [still benefit significantly from it](https://ifs.org.uk/publications/6045), due to marriage between high earners and non-high earners[^2]. Repealing the personal allowance would raise £97 billion (£3,585 per worker) and raise the Gini inequality index from 0.386 to 0.396[^3], indicating that the personal allowance is slightly progressive on a net basis.

[^2]: This and the remaining analysis uses data from 2020, when the personal allowance was set at £12,500.

[^3]: The Gini coefficients used throughout are calculated using total household net income weighted by people.

In [1]:
# Setup
from openfisca_uk import Microsimulation
from openfisca_uk import *
from openfisca_core.model_api import Reform
from openfisca_uk.entities import Person, BenUnit, Household
from openfisca_core.model_api import *
from openfisca_uk.tools.general import *
from ubicenter import format_fig
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
#change "browser" to "notebook" when working on non-ines computers

sim = Microsimulation(year=2020)

from openfisca_core import periods
def make_PA_reform(PA_amount):
    
    def update_PA_parameter(parameters):
        parameters.tax.income_tax.allowances.personal_allowance.amount.update(period=periods.period("year:2020:1"), value=PA_amount)
        return parameters
    
    class reform(Reform):
        def apply(self):
            self.modify_parameters(update_PA_parameter)
    
    sim_less_PA = Microsimulation(reform, year=2020)

    revenue = sim.calc("net_income").sum()
    revenue_diff = revenue - sim_less_PA.calc("net_income").sum()
    BI_amount = revenue_diff/(sim.calc("people").sum())
    
    class BI(Variable):
        value_type = float
        entity = Person
        label = u"UBI"
        definition_period = YEAR
        def formula(person, period, parameters):
            return(BI_amount)

        
    class gross_income(Variable):
        value_type = float
        entity = Person
        label = u"Gross income, including benefits"
        definition_period = YEAR

        def formula(person, period, parameters):
            COMPONENTS = [
                "employment_income",
                "pension_income",
                "self_employment_income",
                "property_income",
                "savings_interest_income",
                "dividend_income",
                "miscellaneous_income",
                "benefits",
                "BI"
            ]
            return add(person, period, COMPONENTS)   

    class basic_income(Reform):
        def apply(self):
            self.add_variable(BI)
            self.update_variable(gross_income)
            
    sim_BI = Microsimulation(reform, basic_income, year=2020)

    return reform, basic_income

/Users/inesfernandez/opt/anaconda3/envs/openfisca/lib/python3.7/site-packages/openfisca_core/parameters/config.py:17: LibYAMLWarning: libyaml is not installed in your environment. This can make OpenFisca slower to start. Once you have installed libyaml, run 'pip uninstall pyyaml && pip install pyyaml --no-cache-dir' so that it is used in your Python environment.

  warnings.warn(" ".join(message), LibYAMLWarning)


In [2]:
#Only run once 
PA_amounts2 = [0, 500, 1500, 2500, 3500, 4500, 5500, 6500, 7500, 8500, 9500, 10500, 11500, 12500]
reform_list = [make_PA_reform(i) for i in PA_amounts2]

In [3]:
sim_list = [Microsimulation(reform, year=2020) for reform in reform_list]

In [4]:
sim_BI = sim_list[0]
PA_amounts = [12500, 11500, 10500, 9500, 8500, 7500, 6500, 5500, 4500, 3500, 2500, 1500, 500, 0]
PA_amounts2 = [0, 500, 1500, 2500, 3500, 4500, 5500, 6500, 7500, 8500, 9500, 10500, 11500, 12500]
income = sim.calc("equiv_household_net_income")

In [34]:
#Income % change for deciles w vs w/o PA
class PA_reform(Reform):
    def apply(self):
        self.neutralize_variable("personal_allowance")
        
sim_no_PA = Microsimulation(PA_reform, year=2020)

income_with_PA = sim.calc("equiv_household_net_income") 
income_no_PA = sim_no_PA.calc("equiv_household_net_income")
decile_change = []
for n in range (1, 11):
    #total income difference for the decile
    id = income_with_PA.groupby(income.decile_rank()).sum()[n] - income_no_PA.groupby(income.decile_rank()).sum()[n]
    #average income difference for the decile
    id2 = id/income_with_PA.groupby(income.decile_rank()).count()[n]
    #average income difference of the decile as a percent of average income for the decile
    id3 = id2/income_with_PA.groupby(income.decile_rank()).mean()[n] * 100
    decile_change.append(id3.round(2))

change = pd.Series(decile_change)
chart1 = format_fig((px.bar(y=decile_change, labels={"x":"Decile", "y":"Income gain"}).update_layout(
    title_text='Income increase brought by the personal allowance for households in each income decile',
    xaxis_title ="Household income decile",
    yaxis_title ="Change in household net income",
    showlegend= False,
    yaxis_ticksuffix="%",
    xaxis = dict(
        tickvals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        ticktext = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    ),
    
).update_traces(marker_color='#1976D2')))
chart1

The benefit for low earners is also constrained by its interaction with means tested benefits. Since Universal Credit phases out with respect to post-tax income, any tax reduction also reduces Universal Credit payments. As [explained](https://www.politics.co.uk/opinion-former/press-release/2018/10/29/personal-allowance-increase-does-little-for-those-on-lowest-income/) by Victoria Todd, Head of the Low Income Tax Reform Group, regarding the 2019 changes:

>“[Universal credit recipients] will not see the full tax gain of £130 from the increase in the personal allowance; instead, they will only gain overall by £48.10, as their Universal Credit award will be reduced by £81.90. However, those earning above £11,850 who receive tax credits will benefit from the full £130 because tax credits are based on gross income.”

Similarly, the [Resolution Foundation](https://www.resolutionfoundation.org/app/uploads/2014/12/Missing-the-target1.pdf) found that the Liberal Democrats’ plan to increase the personal allowance from £10,600 in 2015 to £12,500 by 2020 gave only around £18 of additional annual income to the bottom 10% of households and £203 to the top 10%.

### Replacing the personal allowance with UBI

While the personal allowance is only slightly progressive by itself, using revenue from repealing the personal allowance to fund a UBI is highly progressive. Fully repealing the Personal Allowance could fund a weekly UBI of £29 per person, and this lowers the Gini index from 0.386 to 0.369.

Each £2,000 decrease in the personal allowance could fund an increase in the UBI amount of around £110 annually, or just over £2 a week.

In [44]:
#UBI amount at different levels of PA (graph + calc)

def find_ubi_amount(PA_amount):
    
    def update_PA_parameter(parameters):
        parameters.tax.income_tax.allowances.personal_allowance.amount.update(period=periods.period("year:2020:1"), value=PA_amount)
        return parameters
    
    class reform(Reform):
        def apply(self):
            self.modify_parameters(update_PA_parameter)
    
    sim_less_PA = Microsimulation(reform, year=2020)
    revenue = sim.calc("net_income").sum()
    revenue_diff = revenue - sim_less_PA.calc("net_income").sum()
    BI_amount = (revenue_diff/(sim.calc("people").sum()))/52
    
    return BI_amount.round()

UBI_amounts = [find_ubi_amount(i) for i in (PA_amounts)]


PA_text = ["12500", "11500", "10500", "9500", "8500", "7500", "6500", "5500", "4500", "3500", "2500", "1500", "500", "0"]
chart = format_fig(px.line(x=PA_text, y=UBI_amounts, labels={"x": "Personal allowance amount", "y":"UBI amount"}).update_layout(
    title_text='Amount of UBI at different levels of personal allowance reduction',
    xaxis_title ="Amount of personal allowance (£)",
    yaxis_title ="UBI (£)",
).update_yaxes(range=[-0.1, 30]).add_hline(y=0, line_dash="dash", line_color="#BDBDBD").add_vline(x=0, line_dash="dash", line_color="#BDBDBD"))


This policy would, in general, serve as an income transfer from the richest 40% to the poorest 60%. You can adjust the personal allowance amount with the slider to see how partial personal allowance replacements fare: with further cuts to the personal allowance and higher universal basic incomes, reforms achieve stronger redistributive effects.


In [37]:
def decile_diff_ag(sim_BI):
    gains_by_decile = []
    x2 = sim_BI.calc("household_net_income") - sim.calc("household_net_income")
    for n in range(1, 11):
        values = x2[income.decile_rank() == n]
        mean = values.sum()/values.count()
        gains_by_decile.append(mean.round())
    return gains_by_decile

In [38]:
#Decile impact 
df_list = []
sim_num = 0
deciles = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in PA_amounts2:
    df = pd.DataFrame({"decile": deciles[1:], "Change in income": decile_diff_ag(sim_list[sim_num]), "PA amount": i})
    df_list.append(df)
    sim_num += 1
    
final_df = pd.concat(df_list)
format_fig(px.bar(final_df, x="decile", y="Change in income", animation_frame="PA amount", range_y=[-2500,2500],
                 hover_data={"PA amount": False, "decile": False}).update_layout(
    title_text='Effect of replacing personal allowance with UBI on net income by decile',
    xaxis_title ="Equivalized household net income decile",
    yaxis_title ="Change in household net income",
    showlegend= False,
    yaxis_tickprefix="£",
).update_traces(marker_color='#1976D2'))

For the three lowest deciles, this would mean an annual household income increase of 24%, 11%, and 7% respectively. In contrast, the income reductions for the highest deciles don’t exceed 3%.


In [46]:
#Percentage version
def decile_diff_ag_percent(sim_BI):
    for n in range(1, 11):
        total = sim.calc("household_net_income")[income.decile_rank()==n].sum()
        return ((decile_diff_ag(sim_BI)/income.groupby(income.decile_rank()).mean()) * 100).round()
    
df_list = []
sim_num = 0
for i in PA_amounts2:
    df = pd.DataFrame({"decile": deciles[1:], "Change in income": decile_diff_ag_percent(sim_list[sim_num]), "PA amount": i})
    df_list.append(df)
    sim_num += 1
    
final_df = pd.concat(df_list)
format_fig(px.bar(final_df, x="decile", y="Change in income", animation_frame="PA amount", range_y=[-4,25],
                 hover_data={"PA amount": False, "decile": False}).update_layout(
    title_text='Effect of replacing personal allowance with UBI on net income by decile',
    xaxis_title ="Equivalized household net income decile",
    yaxis_title ="Percentage change in household net income",
    showlegend= False,
    yaxis_ticksuffix="%",
).update_traces(marker_color='#1976D2'))

Additionally, each £1000 reduction in personal allowance buys a 0.0015 reduction in the Gini coefficient.

In [43]:
#gini graph
sim_num = 0
ginis = []
for i in PA_amounts2:
    gini = sim_list[sim_num].calc("household_net_income", map_to="person").gini()
    ginis.append(gini.round(4))
    sim_num += 1
ginis

gini2 = pd.Series(ginis)
chart1 = format_fig((px.line(x=PA_amounts2, y=gini2, labels={"x":"Personal allowance", "y":"Gini coefficient"}).update_layout(
    title_text='Gini coefficient at different levels of personal allowance',
    xaxis_title ="Personal allowance",
    yaxis_title ="Gini coefficient",
    showlegend= False,
).update_yaxes(range=[0.3689, 0.3861]).add_hline(y=0.386, line_dash="dash", line_color="#BDBDBD").add_vline(x=0, line_dash="dash", line_color="#BDBDBD")).update_traces(marker_color='#1976D2'))
chart1

Median household income would increase by about £525, and this number is over five times greater for those living in poverty. This is an income gain of 1.4% for the overall population, 21% for people in poverty, and 53% for people in deep poverty.

In [50]:
#Median difference in household net income for different poverty groups 
isDeepPoor = sim.calc("in_deep_poverty_bhc", map_to ="person")
isPoor = sim.calc("in_poverty_bhc", map_to = "person") & ~isDeepPoor
df_list = []
sim_num = 0
poverty_types = ["Deep poverty", "Poverty, not deep", "Overall"]

def find_DP(sim_BI):
    effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    return effect[isDeepPoor].median().round()

def find_pov(sim_BI):
    effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    return effect[isPoor].median().round()

def find_overall(sim_BI):
    effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    return sim_BI.calc("household_net_income").median().round() - sim.calc("household_net_income").median().round()

for i in PA_amounts2:
    changes = pd.Series([find_DP(sim_list[sim_num]), find_pov(sim_list[sim_num]), find_overall(sim_list[sim_num])])
    df = pd.DataFrame({"Poverty group": poverty_types, "Income gain": changes, "PA amount": i})
    df_list.append(df)
    sim_num += 1


final_df = pd.concat(df_list)
final_df
format_fig(px.bar(final_df, x="Poverty group", y="Income gain", animation_frame="PA amount", range_y=[0,3500],
                 hover_data={"PA amount": False, "Poverty group": False}).update_layout(
    title_text='Change to net income from replacing personal allowance with UBI by poverty status',
    xaxis_title ="",
    yaxis_title ="Median gain in household net income",
    yaxis_tickprefix="£",
    ).update_traces(marker_color='#1976D2'))


Part of the large gains for poor households owe to the fact that poor households tend to have more children, and therefore receive more UBI payments. Similarly, households in deep poverty tend to have fewer children than other poor households, which explains why the median gain is slightly lower for this group.


In [12]:
#Average children per household for each poverty group
isDeepPoor = sim.calc("in_deep_poverty_bhc")
isPoor = sim.calc("in_poverty_bhc") & ~isDeepPoor
isNotPoor = ~isPoor & ~isDeepPoor

poor_kids = sim.calc("num_children", map_to="household")[isPoor].mean()
deep_poor_kids = sim.calc("num_children", map_to="household")[isDeepPoor].mean()
not_poor_kids = sim.calc("num_children", map_to="household")[isNotPoor].mean()
format_fig(px.bar(x=["Deep poverty", "In poverty, but not deep", "Not in poverty"], y=[deep_poor_kids.round(2), poor_kids.round(2), not_poor_kids.round(2)],
                 labels={"y":"Average children", "x": "Household type"}).update_layout(
    title_text="Children per household by poverty status",
    yaxis_title="Average number of children",
    xaxis_title="").update_traces(marker_color='#1976D2'))

Overall, since households with children receive more UBI payments, they are disproportionately benefited.

In [13]:
#Mean income difference grouped by family type (scrollable)

family_type = pd.Series(["Couple without children", "Couple with children", "Single parent", "Single individual"])
df_list = []
sim_num = 0
def find_fam_diff(sim_BI): 
    person_income_diff = sim_BI.calc("net_income") - sim.calc("net_income")
    family = sim.calc("family_type", map_to = "person")
    income_diff_by_family = person_income_diff.groupby(family).mean().reset_index(drop = True)
    return income_diff_by_family.round()

for i in PA_amounts2:
    df = pd.DataFrame({"Family type": family_type, "Income change": find_fam_diff(sim_list[sim_num]), "PA amount": i})
    df_list.append(df)
    sim_num += 1

final_df = pd.concat(df_list)
final_df
format_fig(px.bar(final_df, x="Family type", y="Income change", animation_frame="PA amount", range_y=[-750,1100],
                 hover_data={"PA amount": False, "Family type": False}).update_layout(
    title_text='Average change in net income grouped by family type',
    yaxis_title="Change in net income",
    xaxis_title="",
    yaxis_tickprefix="£"
    ).update_traces(marker_color='#1976D2'))

In fact, this policy would benefit children more than any other group, as children do not lose income to a personal allowance decrease, and exclusively gain from the UBI amount given to them. Child poverty would fall 50%, and deep child poverty by 72%. 

Overall poverty would fall by 29% and overall deep poverty would fall by 44%. In contrast, UBI funded through only partial reductions in the personal allowance would produce smaller poverty impacts than a full replacement. Each £2,000 of personal allowance reduced buys around a 5% decrease in poverty, meaning each £2,000 of personal allowance preserved decreases the poverty effect by 5 percentage points.

In [14]:
#pov reducs at different levels of PA (calculations)
def find_poverty_diff(sim_BI):
    poverty_difference = sim.calc("in_poverty_bhc", map_to="person").sum() - sim_BI.calc("in_poverty_bhc", map_to="person").sum()
    percentage_difference = (poverty_difference/sim.calc("in_poverty_bhc", map_to="person").sum()) * 100
    return -percentage_difference

PA_amounts = [12500, 11500, 10500, 9500, 8500, 7500, 6500, 5500, 4500, 3500, 2500, 1500, 500, 0]
UBI_amounts_text=["UBI=0", "UBI=2", "UBI=4", "UBI=6", "UBI=8", "UBI=10", "UBI=12", "UBI=15", "UBI=17", "UBI=20", "UBI=22", "UBI=25", "UBI=27",
                 "UBI=29"]
sim_num = 0
pov_diffs = []
for i in PA_amounts2:
    diff = find_poverty_diff(sim_list[sim_num]).round()
    pov_diffs.append(diff)
    sim_num += 1

df1 = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Poverty difference": pd.Series(pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})

def find_deep_poverty_diff(sim_BI):
    deep_poverty_difference = sim.calc("in_deep_poverty_bhc", map_to="person").sum() - sim_BI.calc("in_deep_poverty_bhc", map_to="person").sum()
    percentage_difference = (deep_poverty_difference/sim.calc("in_deep_poverty_bhc", map_to="person").sum()) * 100
    return -percentage_difference

sim_num = 0
deep_pov_diffs = []
for i in PA_amounts2:
    diff = find_deep_poverty_diff(sim_list[sim_num]).round()
    deep_pov_diffs.append(diff)
    sim_num += 1

df2 = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Deep poverty difference": pd.Series(deep_pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})


In [15]:
#pt2
def find_child_pov(sim_BI):
    poor_kids_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_child")].sum()
    poor_kids_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_child")].sum()
    child_pov_reduction = (((poor_kids_before - poor_kids_after)/poor_kids_before) * -100).round()
    return child_pov_reduction

sim_num = 0
child_pov_diffs = []
for i in PA_amounts2:
    diff = find_child_pov(sim_list[sim_num]).round()
    child_pov_diffs.append(diff)
    sim_num += 1

df3 = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Child poverty difference": pd.Series(child_pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})


def find_adult_pov(sim_BI):
    poor_adults_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_adult")].sum()
    poor_adults_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_adult")].sum()
    adult_pov_reduction = (((poor_adults_before - poor_adults_after)/poor_adults_before) * -100).round()
    return adult_pov_reduction

sim_num = 0
adult_pov_diffs = []
for i in PA_amounts2:
    diff = find_adult_pov(sim_list[sim_num]).round()
    adult_pov_diffs.append(diff)
    sim_num += 1

df4 = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Adult poverty difference": pd.Series(adult_pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})

def find_child_deep_pov(sim_BI):
    poor_kids_before = sim.calc("in_deep_poverty_bhc", map_to="person")[sim.calc("is_child")].sum()
    poor_kids_after = sim_BI.calc("in_deep_poverty_bhc", map_to="person")[sim_BI.calc("is_child")].sum()
    child_pov_reduction = (((poor_kids_before - poor_kids_after)/poor_kids_before) * -100).round()
    return child_pov_reduction

sim_num = 0
child_deep_pov_diffs = []
for i in PA_amounts2:
    diff = find_child_deep_pov(sim_list[sim_num]).round()
    child_deep_pov_diffs.append(diff)
    sim_num += 1

df5 = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Child deep poverty difference": pd.Series(child_deep_pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})

def find_adult_deep_pov(sim_BI):
    poor_adults_before = sim.calc("in_deep_poverty_bhc", map_to="person")[sim.calc("is_adult")].sum()
    poor_adults_after = sim_BI.calc("in_deep_poverty_bhc", map_to="person")[sim_BI.calc("is_adult")].sum()
    adult_pov_reduction = (((poor_adults_before - poor_adults_after)/poor_adults_before) * -100).round()
    return adult_pov_reduction

sim_num = 0
adult_deep_pov_diffs = []
for i in PA_amounts2:
    diff = find_adult_deep_pov(sim_list[sim_num]).round()
    adult_deep_pov_diffs.append(diff)
    sim_num += 1

df6 = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Adult deep poverty difference": pd.Series(adult_deep_pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})


In [16]:
#pov rates at different levels of PA (graph)

import plotly.graph_objects as go
chart8 = format_fig(go.Figure(data=[
    go.Scatter(name='Poverty', x=df1["PA amounts"], y=df1["Poverty difference"], hovertext=df1["UBI amount"], marker_color='#49A6E2', mode='lines'),
    go.Scatter(name='Deep poverty', x=df2["PA amounts"], y=df2["Deep poverty difference"], hovertext=df2["UBI amount"], marker_color="#0F4AA1", mode='lines'),
    go.Scatter(name='Child poverty', x=df3["PA amounts"], y=df3["Child poverty difference"], hovertext=df3["UBI amount"], mode='lines'),
    go.Scatter(name='Adult poverty', x=df4["PA amounts"], y=df4["Adult poverty difference"], hovertext=df4["UBI amount"], mode='lines'),
    go.Scatter(name='Child deep poverty', x=df5["PA amounts"], y=df5["Child deep poverty difference"], hovertext=df5["UBI amount"], mode='lines'),
    go.Scatter(name='Adult deep poverty', x=df6["PA amounts"], y=df6["Adult deep poverty difference"], hovertext=df6["UBI amount"], mode='lines'),
    
    
], layout_yaxis_range=[-75,0.15]).update_layout( 
    title_text='Poverty decreases at different levels of personal allowance', 
    xaxis_title ="Personal allowance amount",
    yaxis_title ="Decrease in poverty",
    yaxis_ticksuffix="%").add_hline(y=0, line_dash="dash", line_color="#BDBDBD").add_vline(x=12500, line_dash="dash", line_color="#BDBDBD")
                   )

On average, the policy would serve as a transfer from older adults to children. The average child under 18 would see their household net income rise by £2,622 under full replacement of the personal allowance with a UBI.  


In [17]:
#Mean difference in household income by age (calc)
ages = []
for i in range(81):
    ages.append(i)
ages = pd.Series(ages)

def income_by_age(sim_BI):
    gains_by_age = []
    x1 = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    for n in range(81):
        values = x1[age.round() == n]
        mean = values.sum()/values.count()
        gains_by_age.append(mean)
    return gains_by_age

In [18]:
#Mean difference in household income by age (graph)
df_list = []
sim_num = 0
age = sim.calc("age", map_to ="person")
for i in PA_amounts2:
    df = pd.DataFrame({"Age": ages, "Income change": income_by_age(sim_list[sim_num]), "PA amount": i})
    df_list.append(df)
    sim_num += 1
    
final_df = pd.concat(df_list)
final_df
format_fig(px.bar(final_df, x="Age", y="Income change", animation_frame="PA amount", range_y=[-1100,3100],
                 hover_data={"PA amount": False}).update_layout(
    title_text='Average change in household net income by age',
    yaxis_title="Change in household net income",
    yaxis_tickprefix="£",
    xaxis_title="Age",
    showlegend= False,
    ).update_traces(marker_color='#1976D2'))

#### Impacts for specific household types

Overall, 52% of people come out ahead, most of them belonging to the bottom four deciles. 91% of the lowest decile, 73% of the second-lowest decile, and 62% of the third-lowest decile would see their incomes rise by over 5%. In contrast, most people who see their income decrease are in the upper deciles and lose less than 5%.


In [19]:
from openfisca_uk.api import graphs
reform_names = PA_amounts2

In [20]:
#Inter-decile graph
reform_sims = reform_list
baseline = sim
sims = sim_list
NAMES = (
"Gain more than 5%",
"Gain less than 5%",
"No change",
"Lose less than 5%",
"Lose more than 5%"
)

def intra_decile_graph_data(baseline, *reform_sims):
    AMOUNTS = PA_amounts2
    l = []
    for i, reform_sim in enumerate(reform_sims):
        income = baseline.calc("equiv_household_net_income", map_to="person")
        decile = income.decile_rank()
        gain = reform_sim.calc("household_net_income", map_to="person") - baseline.calc("household_net_income", map_to="person")
        rel_gain = (gain / baseline.calc("household_net_income", map_to="person")).dropna()
        bands = (None, 0.05, 1e-3, -1e-3, -0.05, None)
        for upper, lower, name in zip(bands[:-1], bands[1:], NAMES):
            fractions = []
            for j in range(1, 11):
                subset = rel_gain[decile == j]
                if lower is not None:
                    subset = subset[rel_gain > lower]
                if upper is not None:
                    subset = subset[rel_gain <= upper]
                fractions += [subset.count() / rel_gain[decile == j].count()]
            tmp = pd.DataFrame({"PA": f"£{AMOUNTS[i]}", "fraction": fractions, "decile": list(range(1, 11)), "Outcome": name})
            l.append(tmp)
    return pd.concat(l).reset_index()

intra = intra_decile_graph_data(baseline, *sims)

GREY = "#BDBDBD"

COLORS = ("#616161", GREY, "#F5F5F5", "#C5E1A5", "#558B2F",)[::-1]

intra_graph = format_fig(
    px.bar(
        intra,
        x="fraction",
        y="decile",
        orientation="h",
        color="Outcome",
        animation_frame="PA",
        color_discrete_sequence=COLORS,
    ).update_layout(
        yaxis_tickvals=list(range(1, 11)),
        xaxis_tickformat="%",
        yaxis_title="Decile of household equivalized net income",
        xaxis_title="Share of people",
        title="How replacing the personal allowance with a UBI affects people within deciles",
    )
)
intra_graph

A single individual living alone would see their income rise when they make under £17,000 and over £117,000.


In [22]:
#Calc
budget_single = graphs.budget_chart(reform_list, names=reform_names, variables=["household_net_income", "tax", "benefits"])

In [47]:
#Graph
format_fig(budget_single.update_layout(
    title_text='Effect of replacing the personal allowance with UBI on income for a single person'))

Additionally, the marginal tax rate would rise for single individuals earning lower incomes, and decrease for single individuals making between £100,000 and £125,140—the range in which the personal allowance phases out.


In [24]:
#Calc
mtr_single = graphs.mtr_chart(reform_list, names=reform_names, variables=["household_net_income", "tax", "benefits"])

In [25]:
#Graph
format_fig(mtr_single.update_layout(
    title_text='Marginal tax rate for a single person when replacing the personal allowance with a UBI'))

In contrast, for a couple with two children, household net income always rises regardless of employment income.


In [26]:
def couple_two_children(sim):
    sim.add_person(
        name="adult", age=26, is_household_head=True, is_benunit_head=True
    )
    sim.add_person(name="adult_2", age=27)
    sim.add_person(name="child", age=4)
    sim.add_person(name="child_2", age=6)
    sim.add_benunit(
        adults=["adult", "adult_2"],
        children=["child", "child_2"],
        claims_UC=True,
        claims_legacy_benefits=False,
        claims_child_benefit=True,
    )
    sim.add_household(
        adults=["adult", "adult_2"], children=["child", "child_2"]
    )
    return sim

In [27]:
#Calc
budget_two_kids = graphs.budget_chart(reform_list, names=reform_names, variables=["household_net_income", "tax", "benefits", "child_benefit", "universal_credit"],
                   situation_function=couple_two_children)

In [28]:
#Graph
format_fig(budget_two_kids.update_layout(
    title_text='Effect of replacing the personal allowance with UBI on income for a couple with two kids'))

The effect on their marginal tax rate is similar to that of a single individual, but the rise for those earning lower incomes is less significant.


In [29]:
#Calc
mtr_two_kids = graphs.mtr_chart(reform_list, names=reform_names, variables=["household_net_income", "tax", "benefits"],
                situation_function=couple_two_children)

In [30]:
#Graph
format_fig(mtr_two_kids.update_layout(
    title_text='Marginal tax rate for a single person when replacing the personal allowance with a UBI'))

### Conclusion

My research aligns with other analyses showing the progressivity of replacing the personal allowance with UBI. In 2019, the New Economics Foundation [modeled](https://neweconomics.org/2019/03/nothing-personal) replacing the personal allowance with a budget-neutral Weekly National Allowance paid to adults and an increase in the child benefit, making the payments non-taxable but included in means tests. They found progressive results, stating it would “[lift] 200,000 families out of poverty and shift the £8bn currently spent on tax allowances for the 35% highest income families to the remaining 65% of families”. Compass also [modeled](https://www.compassonline.org.uk/wp-content/uploads/2019/03/Compass_BasicIncomeForAll_2019.pdf) replacing the personal allowance with a budget-neutral flat payment of £25 a week for children and adults of working age, as well as a flat rate citizen’s pension of £164.35. They too found progressive results, with the Gini coefficient decreasing from 0.337 to 0.365. Most recently, [my colleagues found](https://www.ubicenter.org/progressive-adjustments-lib-dem-working-group) that the policy in the Liberal Democrats discussion paper could be made more progressive by repealing all, rather than part, of the personal allowance.

Beyond these other papers, I've found that the impacts occur roughly linearly on the margin, meaning that the policy concept can operate incrementally. Furthermore, the policy disproportionately benefits children, and separate studies show that this will in turn improve educational and health outcomes. The impacts are largely but not uniformly beneficial to low income households, though losses are small.

Given these findings, UBI proponents in the UK should take a serious look at whether keeping the personal allowance is worthwhile, provided the benefits of the UBI that could be funded through eliminating it.